# Calculate O2-O2_bar, T-T_bar, S-S_bar

In [1]:
import Phi_derivative as Pd
import xarray as xr
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']

### calculate (T-T_bar), (O2-O2_bar), (S-S_bar)

In [3]:
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE-FOSI_hindcast_O2_T_S_mean_1948-2017_monthly.nc'
ds_hind = xr.open_dataset(filename)
O2_mean = ds_hind['O2_mean']
T_mean = ds_hind['T_mean']
S_mean = ds_hind['S_mean']
depth = ds_hind['4ddepth']

In [9]:
filename = f'{path}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_monthly.nc'
ds_hind = xr.open_dataset(filename)
O2 = ds_hind['O2']
T = ds_hind['TEMP']
S = ds_hind['SALT']

In [11]:
dO2 = O2 - O2_mean
dT = T - T_mean
dS = S - S_mean

In [12]:
filename = f'{path}/DPLE-FOSI_hindcast_AOU_and_AOU_mean_1948-2017_monthly.nc'
ds_hind = xr.open_dataset(filename)
AOU = ds_hind['AOU']
AOU_mean = ds_hind['AOU_mean']

In [14]:
dAOU = AOU - AOU_mean

In [18]:
dsf = xr.Dataset()
dsf['dO2'] = dO2
dsf['dT'] = dT
dsf['dS'] = dS
dsf['dAOU'] = dAOU

In [19]:
dsf.load()

<xarray.Dataset>
Dimensions:  (z_t: 60, nlat: 384, nlon: 320, time: 840)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    TLAT     (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
    TLONG    (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... 318.9 319.4 319.8
    ULAT     (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.42 72.41 72.41
    ULONG    (nlat, nlon) float64 321.1 322.3 323.4 324.5 ... 319.2 319.6 320.0
  * time     (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
Dimensions without coordinates: nlat, nlon
Data variables:
    dO2      (time, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    dT       (time, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    dS       (time, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    dAOU     (time, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan

In [21]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_dO2_dT_dS_dAOU_1948-2017_monthly.nc', mode='w')

## T_mean, S_mean, O2_mean

In [22]:
filename = f'{path}/DPLE-FOSI_hindcast_O2_T_S_mean_dimension_reduced_1948-2017_monthly.nc'
ds_hind = xr.open_dataset(filename)
O2_mean = ds_hind['O2_mean']
T_mean = ds_hind['T_mean']
S_mean = ds_hind['S_mean']
depth = ds_hind['3ddepth']

## Metabolic Traits Ac, Eo

In [4]:
import traits_db
df = traits_db.df_mi
df.head()

,Species,Phylum,Ao,Ac,Eo
0,Acanthephyra acutifrons,Crustacea,26.809651,13.364732,-0.0278
1,Acanthephyra curtirostris,Crustacea,25.575448,14.364194,0.0380
2,Acanthephyra purpurea,Crustacea,23.310023,7.460879,0.1320
3,Acanthephyra smithi,Crustacea,28.985507,17.657939,-0.0301
5,Acipenser brevirostrum,Chordata,22.123894,4.901716,0.3626


In [5]:
Ac_dist = np.percentile(df.Ac, [50])
#Ao_dist = np.percentile(df.Ao, [25, 50, 75])
Eo_dist = np.percentile(df.Eo, [25, 50, 75])
print(Ac_dist)
#print(Ao_dist)
print(Eo_dist)

[7.34832416]
[0.1985 0.3389 0.5625]


## calculate the partial derivatives

In [23]:
%%time
nrow = len(Ac_dist)
ncol = len(Eo_dist)
from itertools import product
dss = xr.Dataset()

for i, j in product(range(nrow), range(ncol)):
    Ac = Ac_dist[i]
    Eo = Eo_dist[j]
    print(Ac,Eo)
    dss[f'dPhidO2_{nrow*i+j}'] =  Pd.dPhidO2(O2_mean,T_mean,S_mean,depth,Ac,Eo,0.022) #dEodT=0.022
    print('dPhidO2 done')
    dss[f'dPhidT_{nrow*i+j}'] = Pd.dPhidT(O2_mean,T_mean,S_mean,depth,Ac,Eo,0.022)  #dEodT=0.022
    print('dPhidT done')
    dss[f'dPhidS_{nrow*i+j}'] = Pd.dPhidS(O2_mean,T_mean,S_mean,depth,Ac,Eo,0.022)  #dEodT=0.022
    print('dPhidS done')

7.3483241595629805 0.1985
dPhidO2 done
dPhidT done
dPhidS done
7.3483241595629805 0.3389
dPhidO2 done
dPhidT done
dPhidS done
7.3483241595629805 0.5625
dPhidO2 done
dPhidT done
dPhidS done
CPU times: user 1min 50s, sys: 11 s, total: 2min 1s
Wall time: 2min 6s


In [24]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dss.load()

<xarray.Dataset>
Dimensions:    (z_t: 60, nlat: 384, nlon: 320)
Coordinates:
  * z_t        (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    TLAT       (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
    TLONG      (nlat, nlon) float64 320.6 321.7 322.8 ... 318.9 319.4 319.8
    ULAT       (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.42 72.41 72.41
    ULONG      (nlat, nlon) float64 321.1 322.3 323.4 ... 319.2 319.6 320.0
Dimensions without coordinates: nlat, nlon
Data variables:
    dPhidO2_0  (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    dPhidT_0   (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    dPhidS_0   (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    dPhidO2_1  (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    dPhidT_1   (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    dPhidS_1   (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    dPhidO2_2  (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    dPhidT_2   (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    dPhidS_2   (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan

In [25]:
%%time
dss.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_dPhidO2_dPhidT_dPhidS_1948-2017_monthly.nc', mode='w')

CPU times: user 70.9 ms, sys: 84.4 ms, total: 155 ms
Wall time: 518 ms
